In [1]:
import numpy as np
from families import Gaussian, Bernoulli, Poisson, Gamma
from glm import GLM
from simulation import Simulation

import statsmodels.api as sm
import statsmodels

/Users/matthewdrury/anaconda/envs/py36/lib/python3.6/site-packages/matplotlib/__init__.py:913: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))
/Users/matthewdrury/anaconda/envs/py36/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [91]:
N = 100000
X = np.empty(shape=(N, 3))
X[:, 0] = 1.0
X[:, 1] = np.random.uniform(size=N)
X[:, 2] = np.random.uniform(size=N)
nu = 1 - 2*X[:, 1] + X[:, 2]

## Linear Model

In [3]:
y = nu + np.random.normal(size=N)
model = GLM(family=Gaussian())
model.fit(X, y)

In [4]:
model.coef_

array([ 0.99879289, -2.0006803 ,  1.00799456])

In [5]:
model.coef_covariance_matrix_

array([[  6.98622863e-05,  -5.96433071e-05,  -6.01284675e-05],
       [ -5.96433071e-05,   1.19873946e-04,  -2.68603415e-07],
       [ -6.01284675e-05,  -2.68603415e-07,   1.20270128e-04]])

In [6]:
model.coef_standard_error_

array([ 0.00835837,  0.0109487 ,  0.01096677])

In [7]:
mod = sm.OLS(y, X)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.295
Model:                            OLS   Adj. R-squared:                  0.295
Method:                 Least Squares   F-statistic:                 2.088e+04
Date:                Tue, 05 Sep 2017   Prob (F-statistic):               0.00
Time:                        19:34:12   Log-Likelihood:            -1.4185e+05
No. Observations:              100000   AIC:                         2.837e+05
Df Residuals:                   99997   BIC:                         2.837e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.9988      0.008    119.496      0.0

## Run some simulations off the linear model.

In [8]:
s = Simulation(model)

In [9]:
s.sample(X)

array([[-1.86504046,  1.268722  ,  1.53999177, ..., -0.51067277,
         1.90442499, -1.29987712],
       [ 0.96579938, -0.47960584,  2.4245509 , ..., -1.13740984,
        -0.10624268, -1.8308379 ],
       [ 2.53309493,  1.06991928, -1.11712062, ...,  0.10934364,
         0.09572319,  0.21120789],
       ..., 
       [ 0.30771164, -0.71776213, -0.19051014, ...,  0.59630497,
         0.96939723, -1.41683503],
       [ 1.68768969,  1.26198194,  1.61300238, ...,  0.25546412,
         0.42073085, -1.65400626],
       [ 1.61165583,  0.47313103, -2.31639835, ..., -1.21550653,
        -0.95238056,  0.33516321]])

In [10]:
models = s.parametric_bootstrap(X, n_sim=10)
for model in models:
    print(model.coef_)

[ 0.99488002 -2.00089273  1.01513462]
[ 0.99620916 -2.00329799  1.0082082 ]
[ 1.00249626 -1.99777885  1.00764488]
[ 0.99366779 -1.99661951  1.00683727]
[ 1.00650234 -1.99690588  1.00180876]
[ 0.99886632 -2.00427236  1.01953375]
[ 0.98966245 -1.99059115  1.01128558]
[ 1.00805518 -2.0139592   1.00451402]
[ 0.98741245 -1.98724754  1.02018386]
[ 1.00481417 -2.00466512  0.99973427]


In [11]:
models = s.non_parametric_bootstrap(X, y, n_sim=10)
for model in models:
    print(model.coef_)

[ 1.00410295 -2.01294251  1.01355806]
[ 0.98230355 -1.99994868  1.03798303]
[ 0.99167136 -1.9858469   1.01380217]
[ 1.01730049 -2.02949226  1.00084181]
[ 0.99837802 -1.99230047  0.98814525]
[ 1.00178502 -2.00175986  1.00639589]
[ 0.97841083 -1.98361612  1.03346434]
[ 1.00041894 -2.00592022  1.01173244]
[ 1.00953521 -1.99535887  0.99057444]
[ 0.99693852 -2.00361248  1.01571625]


## Linear Model with Sample Weights

In [12]:
sample_weights = np.random.uniform(0, 2, size=N)

In [13]:
model = GLM(family=Gaussian())
model = model.fit(X, y, sample_weights=sample_weights)

In [14]:
model.coef_

array([ 0.99195655, -2.00022866,  1.01954062])

## Logistic Model

In [15]:
p = 1 / (1 + np.exp(-nu))
y_logistic = np.random.binomial(1, p=p, size=N)

In [16]:
model = GLM(family=Bernoulli())
model.fit(X, y_logistic)

In [17]:
model.coef_

array([ 1.01395362, -2.00799614,  0.96839904])

In [18]:
model.dispersion_

array(1.0)

In [19]:
model.coef_covariance_matrix_

array([[  3.26777752e-04,  -2.96457948e-04,  -2.52311438e-04],
       [ -2.96457948e-04,   5.93970379e-04,  -4.18834585e-05],
       [ -2.52311438e-04,  -4.18834585e-05,   5.65291004e-04]])

In [20]:
model.coef_standard_error_

array([ 0.018077  ,  0.02437151,  0.02377585])

In [21]:
mod = sm.Logit(y_logistic, X)
res = mod.fit()
print(res.summary())

Optimization terminated successfully.
         Current function value: 0.623754
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:               100000
Model:                          Logit   Df Residuals:                    99997
Method:                           MLE   Df Model:                            2
Date:                Tue, 05 Sep 2017   Pseudo R-squ.:                 0.06631
Time:                        19:34:13   Log-Likelihood:                -62375.
converged:                       True   LL-Null:                       -66805.
                                        LLR p-value:                     0.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.0140      0.018     56.091      0.000       0.979       1.049
x1            -2.0080      0.

In [23]:
s = Simulation(model)

In [24]:
s.sample(X, n_sim=10)

array([[ 1.,  0.,  1., ...,  1.,  0.,  0.],
       [ 1.,  0.,  1., ...,  1.,  0.,  0.],
       [ 0.,  0.,  0., ...,  1.,  1.,  1.],
       ..., 
       [ 1.,  1.,  0., ...,  1.,  0.,  1.],
       [ 1.,  1.,  0., ...,  1.,  0.,  1.],
       [ 1.,  0.,  0., ...,  1.,  0.,  0.]])

In [25]:
for model in s.parametric_bootstrap(X, n_sim=10):
    print(model.coef_)

[ 1.00960588 -1.97087014  0.94616847]
[ 1.01616533 -2.0104736   0.95113261]
[ 1.02293234 -2.00259354  0.96428846]
[ 1.02402486 -2.01245256  0.96062698]
[ 1.02260423 -1.99837106  0.95870786]
[ 1.00838693 -1.98475785  0.9324539 ]
[ 1.02212694 -2.02612229  0.95564776]
[ 1.02512909 -1.99714821  0.91613927]
[ 1.03128252 -2.02739264  0.94351845]
[ 1.01032915 -2.02339032  0.98839962]


In [26]:
for model in s.non_parametric_bootstrap(X, y_logistic, n_sim=10):
    print(model.coef_)

[ 1.0070959  -1.99626151  0.95368044]
[ 1.03095129 -2.05785138  1.00043045]
[ 1.00865451 -2.00285686  0.98909679]
[ 1.02561903 -2.00117068  0.93433024]
[ 1.0115736  -2.00501628  0.99013534]
[ 1.01166112 -2.03337738  0.98102854]
[ 0.99644654 -1.997615    0.96426644]
[ 0.99481866 -2.02052266  1.00463488]
[ 1.03150949 -2.02061731  0.93648404]
[ 1.00154885 -1.98672047  0.96689192]


## Poission Model

In [27]:
mu = np.exp(nu)
y_poisson = np.random.poisson(lam=mu, size=N)

In [28]:
model = GLM(family=Poisson())
model.fit(X, y_poisson)

In [29]:
model.coef_

array([ 0.99938609, -1.98312159,  0.99121718])

In [30]:
model.coef_covariance_matrix_

array([[  3.44330154e-05,  -2.45732287e-05,  -3.61748057e-05],
       [ -2.45732287e-05,   7.16960402e-05,  -1.43190520e-07],
       [ -3.61748057e-05,  -1.43190520e-07,   6.22668923e-05]])

In [31]:
model.coef_standard_error_

array([ 0.00586797,  0.00846735,  0.00789094])

In [32]:
mod = statsmodels.discrete.discrete_model.Poisson(y_poisson, X)
res = mod.fit()
print(res.summary())

Optimization terminated successfully.
         Current function value: 1.595348
         Iterations 7
                          Poisson Regression Results                          
Dep. Variable:                      y   No. Observations:               100000
Model:                        Poisson   Df Residuals:                    99997
Method:                           MLE   Df Model:                            2
Date:                Tue, 05 Sep 2017   Pseudo R-squ.:                  0.1929
Time:                        19:34:14   Log-Likelihood:            -1.5953e+05
converged:                       True   LL-Null:                   -1.9767e+05
                                        LLR p-value:                     0.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.9994      0.006    170.312      0.000       0.988       1.011
x1            -1.9831      0.

In [33]:
s = Simulation(model)

In [34]:
s.sample(X, n_sim=10)

array([[ 2.,  1.,  3., ...,  1.,  0.,  0.],
       [ 0.,  2.,  2., ...,  1.,  2.,  0.],
       [ 5.,  0.,  0., ...,  2.,  3.,  0.],
       ..., 
       [ 2.,  1.,  2., ...,  1.,  0.,  0.],
       [ 7.,  2.,  4., ...,  3.,  1.,  0.],
       [ 2.,  0.,  1., ...,  1.,  1.,  1.]])

In [35]:
for model in s.parametric_bootstrap(X, n_sim=10):
    print(model.coef_)

[ 0.99981137 -1.99182929  0.99500856]
[ 0.98985653 -1.96167654  0.99367047]
[ 1.00385177 -1.99341872  0.99072473]
[ 1.00123208 -1.98107621  0.98271712]
[ 0.99292567 -1.97883574  1.00215413]
[ 0.99554454 -1.98391779  0.99916083]
[ 0.99838322 -1.98039752  0.99462776]
[ 1.00226912 -1.9775594   0.98646307]
[ 0.99215367 -1.97849685  0.99796345]
[ 1.00284333 -1.98435301  0.98785066]


In [36]:
for model in s.non_parametric_bootstrap(X, y_poisson, n_sim=10):
    print(model.coef_)

[ 1.00082602 -1.99769068  0.99711697]
[ 1.00338357 -1.98280846  0.98449723]
[ 1.00098418 -1.98928944  0.99216284]
[ 0.9943823  -1.97222515  0.99382093]
[ 0.9890319  -1.97326068  1.00179367]
[ 0.99663205 -1.96882272  0.98855223]
[ 0.9964418  -1.99275927  1.00403642]
[ 1.00206384 -1.98658403  0.98754328]
[ 0.99643907 -1.97224465  0.99079267]
[ 0.99527552 -1.97440865  0.99163657]


## Poisson with Exposures

In [51]:
mu = np.exp(nu)
expos = np.random.uniform(0, 10, size=N)
y_poisson = np.random.poisson(lam=(mu*expos), size=N)

In [52]:
model = GLM(family=Poisson())
model.fit(X, y_poisson, offset=np.log(expos))

In [53]:
model.coef_

array([ 0.94549036, -2.05368311,  1.17130459])

In [54]:
model.coef_standard_error_

array([ 0.0636085 ,  0.14539115,  0.1422076 ])

## Gamma Regression

In [108]:
mu = np.exp(nu)
y_gamma = np.random.gamma(shape=2.0, scale=(mu / 2.0), size=N)

In [109]:
gamma_model = GLM(family=Gamma())
gamma_model.fit(X, y_gamma)

In [110]:
gamma_model.coef_

array([ 1.0117957 , -2.00700378,  0.98934236])

In [111]:
gamma_model.coef_standard_error_

array([ 0.00617083,  0.00806251,  0.00807451])

In [112]:
gamma_model.dispersion_

0.54179605862843994

In [113]:
gamma_model = sm.GLM(y_exponential, X, 
                     family=sm.families.Gamma(
                         link=statsmodels.genmod.families.links.log))
res = gamma_model.fit()
print(res.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observations:               100000
Model:                            GLM   Df Residuals:                    99997
Model Family:                   Gamma   Df Model:                            2
Link Function:                    log   Scale:                  0.501003293359
Method:                          IRLS   Log-Likelihood:            -1.3796e+05
Date:                Tue, 05 Sep 2017   Deviance:                       54243.
Time:                        20:10:27   Pearson chi2:                 5.01e+04
No. Iterations:                     5                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.9899      0.006    166.825      0.000       0.978       1.002
x1            -1.9816      0.008   -255.596      0.0

## Exponential Regression

In [115]:
mu = np.exp(nu)
y_exponential = np.random.exponential(scale=mu, size=N)

In [116]:
exponential_model = GLM(family=Gamma())
exponential_model.fit(X, y_exponential)

In [117]:
exponential_model.coef_

array([ 1.01241554, -2.01250481,  0.99741567])

In [118]:
exponential_model.coef_standard_error_

array([ 0.00898469,  0.01173897,  0.01175645])

In [119]:
exponential_model.dispersion_

1.1485656432822708

In [ ]:
gamma_model = sm.GLM(y_exponential, X, 
                     family=sm.families.Gamma(
                         link=statsmodels.genmod.families.links.log))
res = gamma_model.fit()
print(res.summary())

## Linear Model with Correlated Predictors

In [79]:
N = 1000
X = np.empty(shape=(N, 3))
X[:, 0] = 1.0
X[:, 1] = np.random.uniform(size=N)
X[:, 2] = 0.5*X[:, 1] + np.random.uniform(-0.5, 0.5, size=N)
nu = 1 - 2*X[:, 1] + X[:, 2]

In [80]:
y = nu + np.random.normal(size=N)
model = GLM(family=Gaussian())
model.fit(X, y)

In [81]:
model.coef_

array([ 1.15561589, -2.28683053,  0.96661721])

In [82]:
model.coef_covariance_matrix_

array([[  4.29008369e-03,  -6.45504312e-03,   3.82777815e-05],
       [ -6.45504312e-03,   1.61829393e-02,  -6.75995573e-03],
       [  3.82777815e-05,  -6.75995573e-03,   1.36322739e-02]])

In [83]:
model.coef_standard_error_

array([ 0.06549873,  0.12721218,  0.11675733])

In [84]:
mod = sm.OLS(y, X)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.245
Model:                            OLS   Adj. R-squared:                  0.243
Method:                 Least Squares   F-statistic:                     161.6
Date:                Tue, 05 Sep 2017   Prob (F-statistic):           1.64e-61
Time:                        19:41:08   Log-Likelihood:                -1447.3
No. Observations:                1000   AIC:                             2901.
Df Residuals:                     997   BIC:                             2915.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.1556      0.065     17.643      0.0